In [ ]:
import pandas as pd
import os
import random
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torchvision.utils as vutils




In [ ]:
# Hyperparameters

batch_size = 64 # Batch size during training
image_size = 128  # Images are 128x128(resized)
image_channels = 3  # RGB images
ngpu=1 #number of GPUs to use
z_dim = 100  # Size/dimension of the latent vector (input noise)
gen_features = 64  # Feature map size of Generator
dis_features = 64  # Feature map size Discriminator
epochs = 100  # Number of epochs during train
lr = 0.0002  # Learning rate
beta1 = 0.5  # Beta1-A hyperparameter in Adam optimizer
beta2= 0.999 # Beta2-A hyperparameter in for Adam optimizer


In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

Question 1:

Train a DC GAN (with an architecture of your choice) on the given data
with the usual GAN loss. Plot the loss curves for the Generator and
Discriminator losses

Generator for DCGAN

In [ ]:
'''
   Takes input as latent vector(sampled from normal distribution)
   this input passed through series of transpose conv,batch norm and activation function
   layers for upsampling.
   Gives a newly generated images as output
'''

class dcgan_Generator(nn.Module):
    def __init__(self, z_dim, image_channels,gen_features):
        super(dcgan_Generator, self).__init__()
        self.generator_layers=nn.Sequential(
            nn.ConvTranspose2d(in_channels=z_dim,out_channels=gen_features*16,kernel_size=4,stride=1,padding=0,bias=False),
            nn.BatchNorm2d(gen_features*16),
            nn.ReLU(True),

            nn.ConvTranspose2d(in_channels=gen_features*16,out_channels=gen_features*8,kernel_size=4,stride=2,padding=1,bias=False),
            nn.BatchNorm2d(gen_features*8),
            nn.ReLU(True),

            nn.ConvTranspose2d(in_channels=gen_features*8,out_channels=gen_features*4,kernel_size=4,stride=2,padding=1,bias=False),
            nn.BatchNorm2d(gen_features*4),
            nn.ReLU(True),

            nn.ConvTranspose2d(in_channels=gen_features*4,out_channels=gen_features*2,kernel_size=4,stride=2,padding=1,bias=False),
            nn.BatchNorm2d(gen_features*2),
            nn.ReLU(True),

            nn.ConvTranspose2d(in_channels=gen_features*2,out_channels=gen_features,kernel_size=4,stride=2,padding=1,bias=False),
            nn.BatchNorm2d(gen_features),
            nn.ReLU(True),

            nn.ConvTranspose2d(in_channels=gen_features,out_channels=image_channels,kernel_size=4,stride=2,padding=1,bias=False),
            nn.Tanh() #scales pixel values(-1 to 1)

        )


    def forward(self,x):
        return self.generator_layers(x)


Discriminator for DCGAN

In [ ]:
''' Takes an image as input,it would be real or fake(generated).
    this input passed through series of conv,batch norm and activation function
    layers for downsampling
    gives ouput as whether it is real or fake(0 or 1)
'''


class dcgan_Discriminator(nn.Module):
    def __init__(self, image_channels,dis_features):
        super(dcgan_Discriminator, self).__init__()
        self.discriminator_layers = nn.Sequential(
            nn.Conv2d(in_channels= image_channels,out_channels=dis_features,kernel_size=4,stride=2,padding=1,bias=False),
            nn.LeakyReLU(0.2,inplace=True),

            nn.Conv2d(in_channels=dis_features,out_channels=dis_features*2,kernel_size=4,stride=2,padding=1,bias=False),
            nn.BatchNorm2d(dis_features*2),
            nn.LeakyReLU(0.2,inplace=True),

            nn.Conv2d(in_channels=dis_features*2,out_channels=dis_features*4,kernel_size=4,stride=2,padding=1,bias=False),
            nn.BatchNorm2d(dis_features*4),
            nn.LeakyReLU(0.2,inplace=True),

            nn.Conv2d(in_channels=dis_features*4,out_channels=dis_features*8,kernel_size=4,stride=2,padding=1,bias=False),
            nn.BatchNorm2d(dis_features*8),
            nn.LeakyReLU(0.2,inplace=True),

            nn.Conv2d(in_channels=dis_features*8,out_channels=1,kernel_size=4,stride=1,padding=0,bias=False),
            nn.Sigmoid()#gives 0 or 1
        )

    def forward(self,x):
        return self.discriminator_layers(x)


In [ ]:
#Weights initialization
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

In [ ]:
#testing the models(Generator and Discriminator)
def test_models():
    z_dim,k,in_channels,H,W =100,10,3,128,128 #k-examples,we are taking(10)
    x=torch.randn(k,in_channels,H,W)
    discriminator= dcgan_Discriminator(in_channels,64)
    weights_init(discriminator_layers)
    assert discriminator_layers(x).shape==(k,1,1,1)
    generator=dcgan_Generator(z_dim,in_channels,k)
    z=torch.randn((k,z_dim,1,1))
    weights_init(generator)
    assert generator(z).shape==(k,in_channels,H,W)
    print("model working..")

if __name__ == "__main__":
    test_models()

In [ ]:
Gen = dcgan_Generator(z_dim, image_channels, gen_features).to(device)
Gen.apply(weights_init)

Disc = dcgan_Discriminator(image_channels, dis_features).to(device)
Disc.apply(weights_init)

In [ ]:
criterion = nn.BCELoss()

In [ ]:
Disc_optimizer = optim.Adam(Disc.parameters(), lr=lr, betas=(0.5, 0.999))
Gen_optimizer = optim.Adam(Gen.parameters(), lr=lr, betas=(0.5, 0.999))

In [ ]:

Animal_transforms = transforms.Compose(
    [
        transforms.Resize((128, 128)),
        # transforms.RandomHorizontalFlip(),  # Randomly flip images horizontally
        # transforms.RandomVerticalFlip(p=0.5),  # Randomly flip images Vertically
        # transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),  # Translation augmentation
        # transforms.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5),  # Brightness, contrast and saturation
        transforms.ToTensor(),
        transforms.Normalize([0.5] * 3, [0.5] * 3),
    ]
)

Animal_dataset= datasets.ImageFolder(root='/content/drive/MyDrive/chaitanya/animals_resized/animals_resized',transform= Animal_transforms)
dataloader= DataLoader(Animal_dataset,batch_size,shuffle=True)


In [ ]:
# print(f"No.of classes: {len(Animal_dataset.classes)}")
# print(f"No.of images: {len(Animal_dataset)}")
# print("Total Classes:", Animal_dataset.classes)

In [ ]:
from torchvision import datasets, transforms

Butterfly_transforms = transforms.Compose(
    [
        transforms.Resize((128,128)),
        transforms.ToTensor(),
        transforms.Normalize([0.5] * 3, [0.5] * 3),
    ]

)

Butterfly_dataset= datasets.ImageFolder(root='/content/drive/MyDrive/chaitanya/Butterfly_dataset',transform= Butterfly_transforms)
bydataloader= DataLoader(Butterfly_dataset,batch_size,shuffle=True)

In [ ]:
# print(f"No.of classes: {len(Butterfly_dataset.classes)}")
# print(f"No.of images: {len(Butterfly_dataset)}")
# print("Total Classes:", Butterfly_dataset.classes)